In [62]:
## This file is an implememtation of an lipm enviroment
## Author : Avadesh Meduri
## Date : 22/02/2020

import numpy as np
import IPython
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation

In [84]:
'''
This is an inverted pendulum enviromment designed to use for RL. The action in this space is the step length
with respect to the current cop location. 9 possible actions are allowed including 0 step length. 
'''

class LipmEnv:
    def __init__(self, h, max_step_length):
        self.omega = np.sqrt(9.81/h)
        self.max_leg_length = 0.3
        self.dt = 0.001
        self.h = h
        assert (np.linalg.norm([max_step_length, self.h]) < self.max_leg_length)
        self.action_space = np.linspace(-max_step_length, max_step_length, 9) #discretize to 9 actions
        self.A = np.matrix([[1, self.dt], [(self.omega**2)*self.dt, 1]])
        self.B = np.matrix([0, -(self.omega**2)*self.dt])
        self.t = 0
                                     
    def integrate_lip_dynamics(self, x_t, u_t):
        ## integrates dynamics for one time step
        assert np.shape(x_t) == (2,)
        x_t_1 = np.matmul(self.A, np.transpose(x_t)) + np.matmul(self.B.transpose(), [u_t])
        return x_t_1

    def reset_env(self, x0, epi_time):
        ## initialises environment
        assert np.shape(x0) == (2,)
        self.t = 0
        self.sim_data = np.zeros((4, int(epi_time/self.dt)+1))
        assert (np.linalg.norm([x0[0], self.h]) < self.max_leg_length)
        self.sim_data[:,0][0:2] = x0
        self.sim_data[:,0][2] = 0
        self.sim_data[:,0][3] = self.h
        
        return self.sim_data[:,0][0:2]
        
    def step_env(self, u, step_time):
        ## integrates the simulation for the duration of one step time(actual step not time step)
        ## This way we can look at things as snap shots of steps not worrying about things in between
        for i in range(int(step_time/self.dt)):
            self.sim_data[:,self.t + 1][0:2] = self.integrate_lip_dynamics(self.sim_data[:,self.t][0:2],\
                                                       self.sim_data[:,self.t][2])
            self.sim_data[:,self.t + 1][2] = self.sim_data[:,self.t][2] #u0
            self.sim_data[:,self.t + 1][3] = self.sim_data[:,self.t][3] #h
            self.t += 1
        
        self.sim_data[:,self.t][2] += self.action_space[u] # new u
        
        ## modifying state that is returned is such that the origin is u0 instead of the global origin
        ## This ensures that the state x[0] is bounded by the maximum leg size while collecting data
        processed_state = self.sim_data[:,self.t][0:2].copy()
        processed_state[0] -= self.sim_data[:,self.t][2]
        
        return processed_state, self.compute_cost(), self.isdone()
        
    def isdone(self):
        ## checks if the kinematic constraint is violated and terminates episode
        current_leg_length = np.linalg.norm([self.sim_data[:,self.t][0] - self.sim_data[:,self.t][2], self.h])
        if current_leg_length > self.max_leg_length:
            return True
        else:
            return False
    
    def compute_cost(self):
        '''
        computes cost: 
            which is square of distance between u and current com + 
            velocity of Com + square of step length 
        '''
        cost = abs(self.sim_data[:,self.t][0] - self.sim_data[:,self.t][2]) #|x0 - u0|
        cost += abs(self.sim_data[:,self.t][1]) #|xd|
        cost += abs(self.sim_data[:,self.t][2] - self.sim_data[:,self.t-1][2]) # |u|
        
        return cost
    
    def random_action(self):
        ## generates a random action to input to the simulation
        action = np.random.randint(len(self.action_space))
        return action
    
    def show_episode(self, freq, i_no):
        ## Input:
            ## Freq : frame rate (if freq = 5 one in 5 is shown)
            ## i_no : iteration number 
        sim_data = self.sim_data[:,::freq]

        fig = plt.figure()
        ax = plt.axes(xlim=(-2, 2), ylim=(0, sim_data[:,0][3] + 0.05))
        text_str = "iter - " + str(i_no)
        line, = ax.plot([], [], lw=3)
        def init():
            line.set_data([], [])
            return line,
        def animate(i):
            x = sim_data[:,i][0]
            y = sim_data[:,i][3]
            u = sim_data[:,i][2]
            line.set_data([u,x], [0,y])
            return line,
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
        ax.text(0.05, 0.95, text_str, transform=ax.transAxes, fontsize=15,
        verticalalignment='top', bbox=props)
        
        anim = FuncAnimation(fig, animate, init_func=init,
                                       frames=np.shape(sim_data)[1], interval=25, blit=True)

        plt.close(fig)
        plt.close(anim._fig)
        IPython.display.display_html(IPython.core.display.HTML(anim.to_html5_video()))
    

In [86]:
## env test 
run_test = False
if run_test:
    no_steps = 10 ## simulates 10 steps taken by the inverted pendulum
    step_time = 0.1 ## duration after which step is taken by pendulum
    env = LipmEnv(0.2, 0.22)
    state = env.reset_env([0.2,0], no_steps*step_time)
    for t in range(no_steps):
        action = env.random_action()
        next_state, cost, done = env.step_env(8, step_time)
        print(state, action, cost, next_state, done)
        state = next_state

    env.show_episode(5, 1)


[0.2 0. ] 7 1.0910771236987789 [0.03047467 1.06060246] False
[0.03047467 1.06060246] 6 1.5570456988478234 [-0.06716792  1.48987778] False
[-0.06716792  1.48987778] 0 1.6527329516720184 [-0.14304204  1.50969091] False
[-0.14304204  1.50969091] 0 1.3680656364387576 [-0.23592268  1.13214296] True
[-0.23592268  1.13214296] 3 0.5598270787573281 [-0.3930623   0.16676478] True
[-0.3930623   0.16676478] 4 2.569793490224459 [-0.69423107 -1.87556242] True
[-0.69423107 -1.87556242] 8 7.322632064293881 [-1.29221182 -6.03042024] True
[-1.29221182 -6.03042024] 5 16.8952597189663 [ -2.49030702 -14.4049527 ] True
[ -2.49030702 -14.4049527 ] 6 36.14268465515007 [ -4.89617722 -31.24650743] True
[ -4.89617722 -31.24650743] 4 74.82682065091714 [ -9.73003984 -65.09678082] True


Your browser does not support the video tag.